## Car Price Prediction Regression Model
### Project from Machine Learning Bookcamp, Alexey Grigorev
### Copyright 2021, Manning Publications

Written in Python version 3.10.7 <br>
Jupyter Notebook built on Windows 11 local server using VS Code Jupyter Extension v2022.8.1002431955

This project covers the following objectives: 
* Performing exploratory data analysis to identify important features
* Encoding categorical variables for use in ML models
* Using logistic regression with Scikit-learn for classification

The main section of the project primarily follows code from book. Occasional comments, explanations, and a few additions are my own. All code and commentary in "Challenge Problems" section (code block 49 to end) is entirely my own.

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

Data Source: https://www.kaggle.com/datasets/blastchar/telco-customer-churn <br>
downloaded 27 Sep 2022

In [3]:
len(df)

7043

In [4]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [5]:
#check levels in each column
for col in df.columns:
    unique_vals = pd.unique(df[col])
    if len(unique_vals) <= 10:
        print(f'{col}: {unique_vals}')
    else:
        print(f'{col}: Numeric or than 10 levels')

customerID: Numeric or than 10 levels
gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
tenure: Numeric or than 10 levels
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: Numeric or than 10 levels
TotalCharges: Numeric or than 10 levels
Churn: ['No' 'Yes']


In [6]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

There are two columns with problems: SeniorCitizen is type `int` and TotalCharges is of type `object` (when it should be `float`). SeniorCitizen is encoded as 1 or 0 instead of yes and no - this actually works fine for later processing, so we won't change it.

TotalCharges needs to be a `float` variable, however. It is likely that there are non-numeric values somewhere in the column, possibly to represent missing data. We will convert first and then replace all `NA` values with 0 - this way the weight of that column will not be considered for those observations in the resulting model.

In [7]:
total_charges = pd.to_numeric(df.TotalCharges, errors = 'coerce')
df[total_charges.isnull()][['customerID', 'TotalCharges']]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [8]:
#set missing values to 0
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors = 'coerce')
df.TotalCharges = df.TotalCharges.fillna(0)

In the original data, capitalization is inconsistent. To make coding a bit easier, we will convert everything to lower case. We also want to do the same to the categorical data itself, along with converting spaces to underscores.

In [9]:
df.columns = df.columns.str.lower()

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

The target variable `churn` is categorical as well (yes or no), but we want to turn this into an integer (1 or 0).

In [10]:
df.churn = (df.churn == 'yes').astype(int)
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int32

Now, we have processed the data to the point where it's workable. We can split into training and test sets using Scikit-learn.

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
df_train_full, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


We also want a validation set, so we run `train_test_split()` again.

In [13]:
df_train, df_val = train_test_split(df_train_full, test_size = 0.33, random_state = 11)
#note: I would have set test_size to 0.25 (20% of full dataset), but leaving as 0.33 to match expected output

#assign result variable (churn) to separate vectors
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

#remove churn column from input data
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [14]:
#check for null values
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [15]:
#check distribution of target variable
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [16]:
#find proportion of churned customers
global_mean = df_train_full.churn.mean()
global_mean

0.26996805111821087

Next, we need to separate out the categorical and numerical variables. These will be treated differently in future processing.

In [17]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines',
               'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
               'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [18]:
#get number of unique values for categorical columns
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

To try to figure out which categorical variables are the most important, we can compare the distributions of each level with the overall distribution. If the differences between the category means and overall mean are small, that category may not have a strong relationship with the output of the model. If the differences are large, however, that category is more likely to be important to our outcome.

In [19]:
#start by checking distribution of gender variable
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print(f'Female mean: {female_mean}, Male mean: {male_mean}')

Female mean: 0.27682403433476394, Male mean: 0.2632135306553911


Females and males appear to churn at similar overall rates, all else being equal (though we don't account for variance).

In [20]:
#distribution for partner variable
partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print(f'Partner mean: {partner_yes}, No partner mean: {partner_no}')

Partner mean: 0.20503330866025166, No partner mean: 0.3298090040927694


There is a greater difference here. Those without partners (33%) are more likely to churn than those with partners (21%) by a wide margin.

Another way we can account for this is by looking at risk ratio, which is simply the group mean divided by the global mean for negative outcomes.

In [21]:
#this is one area where Python code is less clean than R (w/dplyr) or SQL equivalents
df_group = df_train_full.groupby(by = 'gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean

df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [22]:
#now looping through all categorical variables

from IPython.display import display

for col in categorical:
    df_group = df_train_full.groupby(by = col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


We see that there are fairly significant differences between the categories in likelihood of churning. Some key points:

* Gender and phone service do not appear to have a major impact.
* Senior citizens tend to churn at just over 1.5 times the rate of the rest of the population.
* People with a partner churn less than those without a partner. Likewise for those with and without dependents.
* Those with fiber optic internet churn at a much higher rate than those with DSL or no internet.
* Clients without tech support tend to churn much more than those who do. Likewise for those without online security. 
* People with month-to-month plans churn at a much higher rate than those with contracts. Those with two year contracts especially have a very low risk of churning.

(Our data only covers a single month. The vast majority of those with contracts would have to pay a penalty to stop service unless that month happened to align with the end of their contract term. If the data covered a longer time range, we would relatively higher churn from those under contract, simply because we would capture more contract terms ending. Month-to-month customers would probably still churn at a higher rate, but we should expect the difference would be less extreme.)

One way we can apply this is through the idea of mutual information. We can measure the degree of dependency between two variables to see how closely linked they are, in this case with the target variable, `churn`. If that dependency is very low, we can safely remove it from the dataset. Conversely, for a variable like `contract`, the dependency should be higher, which will indicate the variables that are important to the model.


In [23]:
from sklearn.metrics import mutual_info_score

#stand-alone function for calculating mutual info - later used in apply call
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending = False).to_frame(name = 'MI')
df_mi

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923
deviceprotection,0.043453
paymentmethod,0.043210
streamingtv,0.031853
streamingmovies,0.031581
paperlessbilling,0.017589


We can see that `contract`, `onlinesecurity`, and `techsupport` give us the most information about `churn`. `gender` and `phoneservice`, as expected, are at the bottom.

For numeric variables, we can calculate the correlation coefficient for similar effect.

In [24]:
df_train_full[numerical].corrwith(df_train_full.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

Both `tenure` and `totalcharges` have a negative correlation with `churn`, meaning `churn` tends to take on a lower value as they get higher. `monthlycharges` is positively correlated, meaning the opposite. `tenure` is the strongest in absolute correlation.

At this point, since we now have a good understanding of all the variables, we move on to feature engineering and model training.

### Feature Engineering

We begin by encoding categorical variables using one-hot encoding. This is the same technique that we used in the car-price-prediction project (https://github.com/mbalexander19/car_price_prediction), but this time we will use Scikit-learn to implement this. This will certainly take *me* less time than writing a function from scratch, as in the last exercise - and it's almost certainly more efficient from a computation perspective too.

In [25]:
train_dict = df_train[categorical + numerical].to_dict(orient = 'records')
train_dict[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'tenure': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9}

In [26]:
#use sklearn DictVectorizer to encode
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse = False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [27]:
#convert dict to matrix
X_train = dv.transform(train_dict)
X_train[0]

array([0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       1.0000e+00, 0.0000e+00, 0.0000e+00, 8.6100e+01, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 7.1000e+01, 6.0459e+03])

We can see from the first row of the data that numerical variables are kept intact, while categorical variables are encoded as 1s and 0s. Below, we can view exactly how DictVectorizer split up the categories.

In [28]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

### Classification Model

We will now build a logistic regression model to categorize whether individuals are or are not churn risks. The logistic regression model will output a value between 0 and 1, with 0 indicating no risk of churn and 1 indicating a certain churn. 

In reality, it's exceedingly unlikely that we will get outputs at either endpoint directly from the regression model. That means we will need to establish a threshold to divide between churn risks and non-churn risks - values above the threshold will be assigned as churn risks in our classification model, and those below will be assigned as non-churn risks. 

The regression output doesn't mean a definite outcome, but it does allow us to target anti-churn actions to those that are most likely to leave. As an example, we might want to offer those people a discount in return for committing to a new contract so that we don't lose their business altogether. We don't necessarily want to offer this directly to everyone though - there is much less benefit business benefit to offering a discount to those willing to stay at full price.

In [29]:
#this is how we would implement a logistic regression function manually
#we're actually going to use sklearn for this, but this shows a basic "under-the hood" implementation

import math

def sigmoid(score):
    return 1 / (1 + math.exp(-score))

def logistic_regression(xi, w, w_0):
    #xi is a vector of features
    #w is a vector of weights
    #w_0 is the linear bias term
    
    for j in range(len(xi)):
        score = w_0 + xi[j] * w[j]
    
    prob = sigmoid(score)
    return prob

In [30]:
#now we train the model using sklearn

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear', random_state = 1)
#random_state is seed, solver is optimization library

model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [31]:
#now we can use the model for validation
#but first we have to run DictVectorizer on the validation data
val_dict = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dict)

#then we can run that data through our trained model
y_pred = model.predict_proba(X_val)
y_pred

array([[0.76509419, 0.23490581],
       [0.73114898, 0.26885102],
       [0.68055085, 0.31944915],
       ...,
       [0.94275122, 0.05724878],
       [0.38477228, 0.61522772],
       [0.93872712, 0.06127288]])

The output actually gives us *two* values for each observation, but it's not hard to see that they are dependent - each row adds to 1. The first column is the probability that customer is not a churn risk, while the second column is the probability that customer *is* a churn risk. Naturally, since the customer must either churn or not, these probabilities add to 1 - meaning we can readily work with either column. (Put more technically, these two events are complementary - $Pr(Churn) : 1 - Pr(Churn^{c})$.)

In [32]:
#select only the second column : Pr(churn risk)
y_pred = y_pred[:, 1]

#array is now one-dimensional
y_pred

array([0.23490581, 0.26885102, 0.31944915, ..., 0.05724878, 0.61522772,
       0.06127288])

In a real situation, we would need to decide on a threshold at this point if we didn't already have one. But, for simplicity, we will assume the threshold here is 0.5. Any customer with a `y_pred` value above the threshold will be classified a churn risk, allowing the customer retention team to take focused action.

In [33]:
churn = y_pred >= 0.5
churn.mean()

0.2478494623655914

About 25% of customers are classified as churn risks by this model. This matches well with the actual mean churn frequency of about 27% (calculated earlier), but we can make a better comparison for each individual value in the validation set.

In [34]:
(y_val == churn).mean()

0.8016129032258065

That means that our model predictions matched the true outcome for about 80% of customers in the validation set.

#### Model Interpretation

We can look more deeply at model output to view the coefficients applied. For the categorical variables, this gives some idea of the relative significance of each (the same is *not* necessarily true of numerical variables because they are not normalized - though a value of 0 tells us a variable is excluded entirely).

In [35]:
#first, the bias term
model.intercept_[0]

-0.12198805628903112

In [36]:
#and then the rest of the coefficients
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.563,
 'contract=one_year': -0.086,
 'contract=two_year': -0.599,
 'dependents=no': -0.03,
 'dependents=yes': -0.092,
 'deviceprotection=no': 0.1,
 'deviceprotection=no_internet_service': -0.116,
 'deviceprotection=yes': -0.106,
 'gender=female': -0.027,
 'gender=male': -0.095,
 'internetservice=dsl': -0.323,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.116,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.168,
 'multiplelines=no_phone_service': 0.127,
 'multiplelines=yes': -0.081,
 'onlinebackup=no': 0.136,
 'onlinebackup=no_internet_service': -0.116,
 'onlinebackup=yes': -0.142,
 'onlinesecurity=no': 0.258,
 'onlinesecurity=no_internet_service': -0.116,
 'onlinesecurity=yes': -0.264,
 'paperlessbilling=no': -0.213,
 'paperlessbilling=yes': 0.091,
 'partner=no': -0.048,
 'partner=yes': -0.074,
 'paymentmethod=bank_transfer_(automatic)': -0.027,
 'paymentmethod=credit_card_(automatic)': -0.136,
 'paymentmethod=electronic_check': 0.175,


To better understand how the model is applied, we can look at what happens when we apply a smaller subset of variables. We will train on just three variables this time, `contract`, `tenure`, and `totalcharges`.

In [37]:
small_subset = ['contract', 'tenure', 'totalcharges']
train_dict_small = df_train[small_subset].to_dict(orient = 'records')
dv_small = DictVectorizer(sparse = False)
dv_small.fit(train_dict_small)

X_small_train = dv_small.transform(train_dict_small)

In [38]:
#view the features used
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'tenure', 'totalcharges'], dtype=object)

`tenure` and `totalcharges` are numeric, so they have not changed in this model. `contract` is still broken down into three categories as in the original model.

In [39]:
#train the model on subset of features
model_small = LogisticRegression(solver = 'liblinear', random_state = 1)
model_small.fit(X_small_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [40]:
#check weights
model_small.intercept_[0]

-0.5772299084462216

In [41]:
dict(zip(dv_small.get_feature_names_out(), model_small.coef_[0].round(3)))

{'contract=month-to-month': 0.866,
 'contract=one_year': -0.327,
 'contract=two_year': -1.117,
 'tenure': -0.094,
 'totalcharges': 0.001}

We can use these coefficients to interpret the model. The bias term, approximately -0.58 means that, in the absence of any other information, the customer is more likely to be retained than churned. A look back at the sigmoid formula tells us why: <br><br> $sigmoid(x) : \frac{1}{1 + e^{-x}}$ <br><br>
Since $e^{0} : 1$, any negative value for x will make the expression in the denominator greater than 2.

Similarly, we can see that one and two year contracts tend to bring the probability of churn down in the model - strongly so, for two year contracts - while month-to-month contracts will increase the probability of churn. Tenure drives output slightly toward retention, while totalcharges have almost no effect.

#### Using the Model

Before we try the model out on outside data, we can run our test set through and see how close we get. 

*(Note: even though Machine Learning Bookcamp splits off a test set early on in this project, it doesn't ever use the `df_test` data. Since we're not doing any model comparison here, we probably could have just skipped creating a second validation set, but I'm going to run the test data through just to see if we get similar results to the validation.)*

In [42]:
#first we have to run DictVectorizer on the validation data
test_dict = df_test[categorical + numerical].to_dict(orient = 'records')
X_test = dv.transform(test_dict)

#then we can run that data through our model
y_pred = model.predict_proba(X_test)[:, 1]

churn = y_pred >= 0.5
churn.mean()

0.23846699787083037

In [43]:
(y_test == churn).mean()

0.8069552874378992

We get a fairly similar output to the validation set - the proportion of customers for whom churn is predicted is slightly lower, but matches are very slightly higher (about 0.5%). The differences are not enough to be significant, so they increase the confidence that our model is around 80% accurate. Since machine learning is a lot like horseshoes and hand grenades\*, that's pretty good for a first try.

\*(in that being close counts)

Now we can try out our model on a random customer.

In [44]:
customer1 = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen' : 0,
    'partner' : 'no',
    'dependents' : 'no',
    'tenure' : 41,
    'phoneservice' : 'yes',
    'multiplelines' : 'no',
    'internetservice' : 'dsl',
    'onlinesecurity' : 'yes',
    'onlinebackup' : 'no',
    'deviceprotection' : 'yes',
    'techsupport' : 'yes',
    'streamingtv' : 'yes',
    'streamingmovies' : 'yes',
    'contract' : 'one_year',
    'paperlessbilling' : 'yes',
    'paymentmethod' : 'bank_transfer_(automatic)',
    'monthlycharges' : 79.85,
    'totalcharges' : 3320.75
}

In [45]:
X_test = dv.transform([customer1])
X_test

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 7.98500e+01, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 4.10000e+01, 3.32075e+03]])

In [46]:
model.predict_proba(X_test)[:,1]

array([0.07332587])

This customer only has a 7% chance of churning. That's far less than our threshold of 50%, so we would not take any further action with this customer.

In [47]:
#second customer
customer2 = {
    'customerid': '1234-abcde', #this is arbitrary and not used by model
    'gender': 'female',
    'seniorcitizen' : 1,
    'partner' : 'no',
    'dependents' : 'no',
    'tenure' : 1,
    'phoneservice' : 'yes',
    'multiplelines' : 'yes',
    'internetservice' : 'fiber_optic',
    'onlinesecurity' : 'no',
    'onlinebackup' : 'no',
    'deviceprotection' : 'no',
    'techsupport' : 'no',
    'streamingtv' : 'yes',
    'streamingmovies' : 'no',
    'contract' : 'month-to-month',
    'paperlessbilling' : 'yes',
    'paymentmethod' : 'electronic_check',
    'monthlycharges' : 85.7,
    'totalcharges' : 85.7
}

In [48]:
X_test = dv.transform([customer2])
model.predict_proba(X_test)[:,1]

array([0.83216369])

This customer has an 83% chance of churning based on our model, so we *would* take action to try to retain her.

## Challenge Problems

### Exercise 1 - Try to redo the car_price_prediction problem from the previous chapter with Scikit-learn.

Response is found in car_price_prediction repository, https://github.com/mbalexander19/car_price_prediction.

### Exercise 2 - We looked at feature importance metrics but didn't really use them. See what happens to the model when some non-useful features (for example, `gender` and `phoneservices`) are excluded. Also see what happens if we exclude the most useful feature (`contract`).

In [49]:
#modify list of categorical vars - numerical vars don't change
categorical_mod = ['seniorcitizen', 'partner', 'dependents', 'multiplelines',
               'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
               'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']

In [50]:
#re-encode variables
train_dict_mod = df_train[categorical_mod + numerical].to_dict(orient = 'records')

#run DictVectorizer
dv = DictVectorizer(sparse = False)
dv.fit(train_dict_mod)

#train model
X_train_mod = dv.transform(train_dict_mod)

model = LogisticRegression(solver = 'liblinear', random_state = 1)
model.fit(X_train_mod, y_train) #target data doesn't change

#re-encode validation set
val_dict_mod = df_val[categorical_mod + numerical].to_dict(orient = 'records')
X_val_mod = dv.transform(val_dict_mod)

#predict values
y_pred = model.predict_proba(X_val_mod)[:,1]


In [51]:
churn = y_pred >= 0.5
churn.mean()

0.2478494623655914

In [52]:
(y_val == churn).mean()

0.8026881720430108

Our model quality with `gender` and `phoneservices` removed is virtually identical to the original model with all variables. That is to be expected - our original analysis indicated these variables don't contribute much to the model.

Now we'll run the model again, with only `contract` removed.

In [53]:
categorical_mod2 = ['gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 'multiplelines',
               'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
               'streamingtv', 'streamingmovies', 'paperlessbilling', 'paymentmethod']

In [54]:
#re-encode variables
train_dict_mod2 = df_train[categorical_mod2 + numerical].to_dict(orient = 'records')

#run DictVectorizer
dv = DictVectorizer(sparse = False)
dv.fit(train_dict_mod2)

#train model
X_train_mod2 = dv.transform(train_dict_mod2)

model = LogisticRegression(solver = 'liblinear', random_state = 1)
model.fit(X_train_mod2, y_train) #target data doesn't change

#re-encode validation set
val_dict_mod2 = df_val[categorical_mod2 + numerical].to_dict(orient = 'records')
X_val_mod2 = dv.transform(val_dict_mod2)

#predict values
y_pred = model.predict_proba(X_val_mod2)[:,1]

In [55]:
churn = y_pred >= 0.5
churn.mean()

0.23709677419354838

In [56]:
(y_val == churn).mean()

0.7973118279569893

Our model quality *still* appear to be nearly identical to the original - within 1% accuracy. This is a bit more surprising than the previous result, but still not terribly unusual. There is enough mutual information between `contract` and the rest of the variables combined that our model can come to the same conclusions even without `contract` being explicitly included.

While we wouldn't necessarily want to exclude `contract` from our current model (we already have the data, after all), but it does mean that our model should remain usable even if the company moves away from the existing contract structure. If, for instance, the company decided to move everyone to a non-contract (effectively month-to-month) payment structure, `contract` probably would no longer be useful (because everyone would have the same kind). But it appears we get enough information from the rest of the variables that we could still use the original model while awaiting new data reflecting the change.